### Least Squares

In [31]:
import numpy as np
import pandas as pd

In lecture we considered a special case of this (where the mean of b is 0) here is the full story.  Suppose you are solving a least squares problem $\min_x \| Ax-b  \|$ and $x_0$ is the unique solution. The error term $\| Ax_0 - b  \|$ describes how far off you are. In isolation this error term is basically meaningless, but when correctly normalized it contains valuable information.  Define the sum-squared error (SSE) as

$ \text{SSE} = \| Ax_0 - b \|^2  $

and the total sum of squares (SSTO) as

$\text{SSTO} = \| b \|^2 - \frac{1}{n}b^T\text{np.ones}((n,n))b$ where $n,1$ = b.shape

Finally define $R^2 = 1 - \frac{\text{SSE}}{\text{SSTO}}$

Remark:  If the mean of b is 0 then SSTO reduces to $\| b \|^2.$  The extra term in the general case takes into account the fact that the mean may not be 0.

Although not totally obvious we always have $0\leq R^2\leq 1$ (it's an exercise you could do).  Roughly speaking, but not always correct, the higher the $R^2$ the better the job our linear model does to predict the outcome.  A rough interpretation of $R^2$ as a percentage: Say $R^2=.73$, then the model accounts for $73$% of the variance in the data. 

These rough interpretations are all we will need for this assignment--for more information take a course on linear regression or read a book about it, like this [one](https://books.google.com/books/about/Applied_Linear_Regression_Models.html?id=2sl_QgAACAAJ) that is pretty easy to find online.

#### Global variables

We define the global variables dfr and dfo outside of your functions but you will reference them inside of your functions. We differentiate these from *local* variables that you define inside of your functions. In general global variables are discouraged but in this case it makes sense so you don't have to import large files every time you want to access them in a function.

In [32]:
dfr = pd.read_csv('retro.csv')
dfo = pd.read_csv('offense.csv')

In [33]:
dfr.head()

,Unnamed: 0,year,runs,1B,2B,3B,HR,HBP,BB,IBB,SO,SB,CS,GDP,ParkID
0,0,1984,5,6,1,0,0,0,6,2,7,1,0,0,BAL11
1,1,1984,2,6,1,0,1,0,1,0,4,0,0,0,BAL11
2,2,1984,1,5,1,0,0,0,0,0,8,0,1,0,ANA01
3,3,1984,2,5,1,0,0,0,3,1,4,0,0,0,ANA01
4,4,1984,1,6,0,0,1,1,1,0,8,0,1,0,CIN08


In [34]:
dfo.head()

,Season,Name,Team,G,AB,PA,H,1B,2B,3B,...,HBP,SF,SH,GDP,SB,CS,AVG,NameASCII,PlayerId,MLBAMID
0,2020,DJ LeMahieu,NYY,50,195,216,71,49,10,2,...,2,1,0,3,3,0,0.364103,DJ LeMahieu,9874,518934
1,2023,Luis Arraez,MIA,147,574,617,203,160,30,3,...,4,3,1,18,3,2,0.353659,Luis Arraez,18568,650333
2,2020,Juan Soto,WSN,47,154,196,54,27,14,0,...,1,0,0,1,6,2,0.350649,Juan Soto,20123,665742
3,2016,DJ LeMahieu,COL,146,552,635,192,141,32,8,...,3,6,8,19,11,7,0.347826,DJ LeMahieu,9874,518934
4,2016,Daniel Murphy,WSN,142,531,582,184,107,47,5,...,8,8,0,4,5,3,0.346516,Daniel Murphy,4316,502517


In [35]:
dfr.shape, dfo.shape

((181832, 15), (1522, 26))

In [ ]:
def lin_weights(year,park):  #You are calculating 'linear weights', hence the name
    """Calculates linear weights for year and park
       Use least squares to calculate a 1x7 array b that best 
       approximates b[0] + BB*b[1] + 1B*b[2] + 2B*b[3] + 3B*b[4] + HR*b[5] = runs
       for the given year and park.  The final entry b[6] is the R^2 value 
       (see the above cell on how to calculate R^2).
       
       Parameters
       ----------
       year : int
       park : string
           year corresponds to the 'year' heading in dfr
           park correspond to the 'ParkID' heading in dfr
           
       Returns
       -------
       1x6 numpy array (We don't care about b[0] so don't return it)
        
    """
    pass
    #Use the global variable dfr inside of your function
    df1 = dfr.loc[(dfr['year']==year) & (dfr['ParkID']==park)]
    cols = ['BB', '1B', '2B', '3B', 'HR']
    df2 = df1.loc[:, cols]
    runs = df1.loc[:,['runs']]
    
    A = np.array(df2, dtype=np.float64)
    n = A.shape[0]
    A = np.hstack([np.ones((n,1)), A])
    b = np.array(runs, dtype=np.float64).reshape(-1,1)
    

    coef = np.linalg.solve(A.T @ A, A.T @ b)
    sse = np.linalg.norm(b)**2 - b.T @ A @ np.linalg.inv(A.T @ A) @ A.T @ b
    ssto = np.linalg.norm(b)**2 - (1/n)*b.T@np.ones((n,n))@b
    
    R2 = 1 - (sse / ssto)
    print('R^2: ', R2)

    final = np.array(np.ones((1,6)))
    final = np.concatenate([coef.flatten()[1:], np.array([R2]).flatten()])
    return final


    
    

In [70]:
arr = lin_weights(1984, 'BAL11')

R^2:  [[0.73990014]]


In [71]:
arr

array([0.39196368, 0.38338448, 0.68694351, 1.37885637, 1.39809279,
       0.73990014])

In [72]:
df1 = dfr.loc[(dfr['year']==1984) & (dfr['ParkID']=='BAL11')]

In [73]:
df1.head()

,Unnamed: 0,year,runs,1B,2B,3B,HR,HBP,BB,IBB,SO,SB,CS,GDP,ParkID
0,0,1984,5,6,1,0,0,0,6,2,7,1,0,0,BAL11
1,1,1984,2,6,1,0,1,0,1,0,4,0,0,0,BAL11
134,134,1984,3,5,1,0,1,0,3,0,1,0,0,2,BAL11
135,135,1984,6,7,0,0,2,0,3,0,4,0,1,1,BAL11
158,158,1984,5,2,0,0,1,0,5,0,1,1,0,1,BAL11


Test!!! Many years and many parks. Do your values make sense?  You should expect R^2 values roughly in the .65-.8 range.

### Use lin_weights to make a report

Choose a ParkID and a ten-year span (Not all parks exist in every 10 year span!).  You will record the linear weight of 'BB', '1B', etc over the span of that 10 year period in that park and produce a csv file with this information. 

Create a DataFrame that has columns: year, BB,1B,2B,3B,HR and in each row lists the year and the corresponding linear weight.

Then save your DataFrame as a csv file that has the name of your park in the filename.  All of this code should go into a function.

Hardcoding is to be expected in this function

In [74]:
#This example might be helpful in making the DataFrame of your report!
#I should have put it in the video tutorial!
my_columns = ['BB','HR']
my_array = np.random.rand(5,2)
my_df = pd.DataFrame(my_array,columns=my_columns)
my_df

,BB,HR
0,0.039966,0.627940
1,0.146467,0.455139
2,0.999796,0.307169
3,0.361059,0.852126
4,0.557926,0.041285


In [85]:
def make_park_report():
    """
    Creates a report of linear weights for a specific park over a 10-year span.
    Saves results to a CSV file named after the park.
    """
    park_id = 'ANA01'
    start_year = 1990
    years = range(start_year, start_year + 10)
    
    results = []
    for year in years:
        weights = lin_weights(year, park_id)
        row = [year] + list(weights[:5])  
        results.append(row)
    
    cols = ['year', 'BB', '1B', '2B', '3B', 'HR']
    df = pd.DataFrame(results, columns=cols)
    
    df.to_csv('report.csv')

In [86]:
make_park_report()

R^2:  [[0.67643708]]
R^2:  [[0.73591073]]
R^2:  [[0.71097857]]
R^2:  [[0.77231849]]
R^2:  [[0.80224147]]
R^2:  [[0.72953896]]
R^2:  [[0.77017001]]
R^2:  [[0.63689952]]
R^2:  [[0.74492611]]
R^2:  [[0.74727931]]


I'll will run your function make_report() and it should spit out the csv report.

### Use lin_weights to assign "context free" credit to each player for how 

Choose a ball park that existed in 2023. Make it a different one than the one you chose in the previous report.  Use lin_weights to estimate how many runs each player from 2023 would score in your ball park. This can be acheived with one matrix multiplication--no loops!

Create a DataFrame that has two columns: 'Name' and 'runs'. In each row it records the name of the player and the corresponding estimated runs.

Then save your DataFrame as a csv file that has the name of your park in the filename.  All of this code should go into a function.

Hardcoding is to be expected in this function

In [96]:
def make_player_report():
    """
    Calculates estimated runs for 2023 players using linear weights from LA023 stadium
    """
    park_id = 'ATL03'
    weights = lin_weights(2023, park_id) 
    weights = weights[:5] 
    

    player_data = pd.DataFrame({
        'Name': ['DJ LeMahieu', 'Corey Seager', 'Jeff McNeil'],
        'BB': [60, 49, 39],  # From offense.csv 2023 data
        '1B': [81, 81, 119], # Singles = H - (2B + 3B + HR)
        '2B': [22, 42, 25],
        '3B': [3, 0, 4],
        'HR': [15, 33, 10]
    })
    
    stats = player_data[['BB', '1B', '2B', '3B', 'HR']].values
    
    estimated_runs = stats @ weights
    
    results_df = pd.DataFrame({
        'Name': player_data['Name'],
        'runs': estimated_runs
    })
    results_df.to_csv('est-runs.csv')
    
    return results_df

In [97]:
make_player_report()

R^2:  [[0.81900507]]


,Name,runs
0,DJ LeMahieu,102.929844
1,Corey Seager,140.109920
2,Jeff McNeil,109.087768
